# Book Crossing Dataset Trimming

### Import Packages

In [36]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Load Books Dataset

In [37]:
# Load BX-Books.csv data into memory

df_books = pd.read_csv('Code/DataSet/BX-Books.csv', delimiter=",", error_bad_lines=False, encoding='latin-1', header=0, index_col=False)
df_books.head(5)

,Unnamed: 0,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [38]:
print("Number of Books: {}".format(len(df_books)))

Number of Books: 271360


### Load Users Dataset

In [39]:
# Load BX-Book-Users.csv data into memory

df_users = pd.read_csv('Code/DataSet/BX-Users.csv', delimiter=",", error_bad_lines=False, encoding='latin-1', header=0, index_col=False)
df_users.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [40]:
print("Number of Users: {}".format(len(df_users)))

Number of Users: 278860


### Load Ratings Dataset

In [41]:
# Load BX-Book-Ratings.csv data into memory

df_ratings = pd.read_csv('Code/DataSet/BX-Ratings-Full.csv', delimiter=";", header=0, encoding ='unicode_escape', index_col=False)
df_ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [42]:
print("Number of Ratings: {}".format(len(df_ratings)))

Number of Ratings: 1149780


### Filter to include only records with book ISBNs in Books dataset

In [43]:
#Filter all records with book ISBNs not in df_books dataframe

df_ratings_filtered = df_ratings[df_ratings.ISBN.isin(df_books.ISBN)]

print("Before:", df_ratings.shape[0])
print("After:", df_ratings_filtered.shape[0])

Before: 1149780
After: 1031136


### Filter to include only records with user ID in Users dataset

In [44]:
#Filter all records with user ID not in df_users dataframe

df_ratings_filtered2 = df_ratings_filtered[df_ratings_filtered["User-ID"].isin(df_users["User-ID"])]

print("Before:", df_ratings_filtered.shape[0])
print("After:", df_ratings_filtered2.shape[0])

Before: 1031136
After: 1031136


### Filter to include only explicit ratings i.e. rating from 1 to 10

In [45]:
df_ratings_filtered3 = df_ratings_filtered2[df_ratings_filtered2["Book-Rating"] != 0]

print("Before:", df_ratings_filtered2.shape[0])
print("After:", df_ratings_filtered3.shape[0])

Before: 1031136
After: 383842


### Get top 120 users with the highest number of book ratings

In [46]:
grouped = df_ratings_filtered3.groupby('ISBN')['User-ID'].size().reset_index()
ISBNs = grouped.sort_values('User-ID', ascending=False).head(120)
ISBNs.head(5)

,ISBN,User-ID
21945,0316666343,707
117745,0971880107,581
38819,0385504209,487
18058,0312195516,383
4131,0060928336,320


 ### Get top 120 books with the highest number of user ratings

In [47]:
grouped = df_ratings_filtered3.groupby('User-ID')['ISBN'].size().reset_index()
Users = grouped.sort_values('ISBN', ascending=False).head(120)
Users.head(5)

,User-ID,ISBN
2719,11676,6943
24101,98391,5691
46279,189835,1899
37648,153662,1845
5615,23902,1180


### Filter to include ony records with top 120 users with the highest number of book ratings

In [48]:
df_ratings_users = df_ratings_filtered3[df_ratings_filtered3['User-ID'].isin(Users["User-ID"].tolist())]
df_ratings_users.head(5)         

,User-ID,ISBN,Book-Rating
21689,4385,0061083402,10
21691,4385,0061093343,9
21692,4385,0061096156,10
21694,4385,0312956762,10
21695,4385,0312980353,10


In [49]:
print("Before:", df_ratings_filtered3.shape[0])
print("After:", df_ratings_users.shape[0])

Before: 383842
After: 58571


### Filter to include ony records with top 120 books with the highest number of user ratings

In [50]:
df_ratings_isbn = df_ratings_users[df_ratings_users['ISBN'].isin(ISBNs["ISBN"].tolist())]
df_ratings_isbn.head(5)

,User-ID,ISBN,Book-Rating
21917,4385,0440221471,10
25718,6251,0060392452,10
25744,6251,0061009059,7
25870,6251,0316569321,8
25925,6251,0375725784,4


In [51]:
print("Before:", df_ratings_users.shape[0])
print("After:", df_ratings_isbn.shape[0])

Before: 58571
After: 1052


### Transform Ratings dataset to User-Book Interaction Matrix

In [52]:
ratings_matrix = df_ratings_isbn.pivot(index='User-ID', columns='ISBN', values='Book-Rating')
ratings_matrix.shape

(117, 120)

In [53]:
ratings_matrix.head(5)

ISBN,0060392452,0060502258,0060915544,0060928336,0060930535,0060934417,0060938455,0060959037,0060976845,0060987103,0061009059,006101351X,0064407667,014028009X,0140293248,0140298479,0142000205,0142001740,0156027321,0312195516,0312278586,0312291639,0312305060,0312924585,0312983271,0316096199,0316284955,0316569321,0316601950,0316666343,0316769487,0316776963,0345313860,0345337662,0345339681,0345339703,0345342968,0345353145,0345361792,0345370775,0345417623,0345443284,0375700757,0375706771,0375707972,0375725784,0375726403,0375727345,0380789019,0380789035,0385265700,0385335482,0385484518,0385486804,0385492081,0385504209,0385505833,0385720106,0385722206,0399501487,0439064864,0439064872,0439136350,0439136369,0439139597,0439139600,043935806X,0440206154,044021145X,0440211727,0440213525,0440214041,0440220602,0440221471,044022165X,0440222656,0440224764,0440225701,0440226430,0440234743,0440236673,044023722X,0440241073,0446310786,0446364193,044651652X,0446605239,0446606812,0446608955,0446610038,0446611867,0446672211,0449005615,0449212602,0452282152,0452282829,0553572997,0590353403,059035342X,0671001795,0671003755,0671021001,0671027360,0671510053,0679746048,067976402X,068484477X,0684872153,0684874350,0743237188,0743418174,0786868716,0804106304,0804114986,0805063897,0812550706,0842329129,0971880107,1400034779,1558743669
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6251,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,6.0,NaN,7.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,10.0,NaN,9.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6575,NaN,8.0,NaN,8.0,NaN,9.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,4.0,8.0,6.0,NaN,8.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,5.0,6.0,NaN,NaN,NaN,NaN,7.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,8.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,9.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
7346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,10.0,NaN,9.0,8.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,7.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11676,NaN,8.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,9.0,8.0,8.0,3.0,8.0,5.0,3.0,8.0,NaN,6.0,10.0,6.0,9.0,9.0,NaN,NaN,10.0,NaN,10.0,NaN,5.0,NaN,8.0,8.0,6.0,NaN,NaN,NaN,NaN,10.0,8.0,10.0,NaN,NaN,6.0,9.0,NaN,7.0,7.0,7.0,NaN,7.0,10.0,9.0,NaN,NaN,9.0,8.0,8.0,9.0,NaN,10.0,8.0,NaN,9.0,8.0,10.0,NaN,NaN,1.0,NaN,5.0,NaN,NaN,9.0,NaN,1.0,NaN,8.0,7.0,9.0,8.0,8.0,8.0,NaN,8.0,8.0,6.0,NaN,7.0,10.0,8.0,10.0,10.0,10.0,NaN,10.0,7.0,8.0,10.0,10.0,1.0,NaN,NaN,9.0,NaN,10.0,4.0,9.0,7.0,6.0,8.0,9.0,NaN,8.0,NaN,9.0,9.0,6.0,NaN,NaN


### Save filtered Ratings dataset to file

In [54]:
df_ratings_isbn.to_csv("Code/DataSet/BX-Ratings.csv", index=False)